In [2]:
import pandas as pd

#### SQL

# Script

#### Aux Data

SELECT u.username, u.name, 
um.workforceId, up.shortName as position, ut.name as team_name, tl.name as team_lead,
ua.*, uar.reason 
FROM kjtcallctr.useraction ua 
LEFT JOIN kjt.useractionreason uar ON uar.id = ua.reasonId
LEFT JOIN kjt.user u ON u.uid = ua.uid 
join kjt.usermeta um on um.uid = u.uid
join kjt.userteam ut on utid = um.teamId
join kjt.user tl on tl.uid = ut.teamLead
join kjt.userposition up on up.id = um.positionId 
where up.shortName in ('Tier I CSR','Tier II CSR','Tier III CSR')
and ut.name not in ('NYC Admin','Testing','QFN Agent','MNL Backup')
and ua.createTime > DATE_SUB(CURRENT_TIMESTAMP, INTERVAL 3 DAY);

In [158]:
df = pd.read_csv('Control Tower.csv',parse_dates=['CREATETIME','ENDTIME'],infer_datetime_format=True)
df.columns = [x.lower() for x in df.columns]
df.createtime = df.createtime.dt.tz_localize(None)
df.endtime = df.endtime.dt.tz_localize(None)
df.rename(columns={'reason':'pos_status','createtime':'start_interval_log','endtime':'end_interval_log'}, inplace=True)

/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_47951/103832614.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('Control Tower.csv',parse_dates=['CREATETIME','ENDTIME'],infer_datetime_format=True)


In [159]:
df.pos_status = df.pos_status.fillna('Phone Time')

### Schedule Data

select * from wfm.userschedule
where startTime > DATE_SUB(CURRENT_TIMESTAMP, INTERVAL 3 DAY)
and endTime < DATE_SUB(CURRENT_TIMESTAMP, INTERVAL 1 DAY)


In [160]:
dx = pd.read_csv('pos_schedule.csv',parse_dates=['startTime','endTime'])

In [161]:
dx.columns = [x.lower()  for x in dx.columns]

In [162]:
dx.rename(columns={'starttime': 'start_interval_sched','endtime':'end_interval_sched'}, inplace=True)

#### match names

#### match_agent

##### KEY

1.  Happy
2.  Benefit of the doubt
3. Working When you shouldn't be 
4.  Not Working when you should be 
5. working when you REALLY shouldn't be 
6. Overtime on break
7. Unverified Overtime
8. Happy not working
99. Phone Time == Phone Time

In [163]:
#https://docs.google.com/spreadsheets/d/1_LGtHXwhCCctXMXO_gi_3d9RbjrK3bfzmH8mNFIaMCU/edit#gid=233384588

In [164]:
am  = pd.read_csv('activity_matrix.csv')
am = am.set_index('Unnamed: 0')
activity_matrix = am.to_dict()






#### from here, analyze per agent

In [165]:
def create_scheduled_off(tdx):
    scheduled_offs = []
    for i,row in tdx.loc[tdx.time_between_sched > 0].iterrows():
        row.end_interval_sched = row.start_interval_sched
        row.start_interval_sched = row.last_end_interval_sched
        row['activitytype'] = 'Scheduled Off'
        scheduled_offs.append(row)
    
    return scheduled_offs

In [166]:
def log_exceeds_schedule(dfdirty):
    added_rows = []
    for i,row in dfdirty.iloc[:].iterrows():
        entry1 = row.copy()
        entry1.end_interval_log = entry1.end_interval_sched
        added_rows.append(entry1)
        entry2 = row.copy()
        entry2.start_interval_log = entry2.end_interval_sched
        added_rows.append(entry2)
    df_new = pd.concat([s.to_frame().T for s in added_rows], ignore_index=True)
    df_new = pd.DataFrame(df_new)
    return df_new


In [167]:
def system_issue(category,mins, threshold_mins=5/60,categories=[4,5]):
    if category in categories:
        if mins < threshold_mins:
            return category + 0.1
    else:
        return category
    return category
            

In [210]:
def fix_split_interval(tdf,tdx):
    tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', right_on='start_interval_sched',suffixes=('_log', '_sched'),tolerance = pd.Timedelta('1 hour'), direction='backward')
    
    tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log
    dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.activitytype))]
    dfdirty = tdf1.loc[tdf1.end_surpass]
    for i in [1,2,3]:
        df_new = log_exceeds_schedule(dfdirty)
        df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log
        df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log
        dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])
        dfdirty = df_new.loc[df_new.end_surpass]
    
    dfclean['work_verification_category'] = dfclean.apply(lambda row: activity_matrix.get(row.activitytype,{}).get(row['pos_status'],"No Map"),axis=1)
    dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]
    dfclean['work_verification_category'] = dfclean.apply(lambda row: system_issue(row.work_verification_category,row.mins),axis=1)
    return dfclean

In [211]:
def create_agent_timesheet(workforceid,df=df):
    tdx = dx.loc[dx.workforceid == workforceid]
    tdx = tdx.sort_values(by='start_interval_sched').reset_index(drop=True)
    tdx['last_end_interval_sched']  = tdx['end_interval_sched'].shift(1)
    tdx['time_between_sched'] = [x.seconds / 60 / 60 for x in tdx.start_interval_sched - tdx['end_interval_sched'].shift(1)]
    offs = create_scheduled_off(tdx)
    if len(offs) > 0:
        offs = pd.concat([s.to_frame().T for s in offs], ignore_index=True)
        tdx = pd.concat([tdx,offs])
    else:
        pass
    tdx = tdx.sort_values(by='start_interval_sched',ascending=True)
    tdx.start_interval_sched = pd.to_datetime(tdx.start_interval_sched)

    
    tdf = df.loc[df.workforceid == workforceid]
    dfclean  =  fix_split_interval(tdf,tdx)
    return dfclean

In [212]:
def find_unmapped_matrix(df):
    broken = []
    for i,row in df.iterrows():
        try:
            activity_matrix.get(row.activitytype).get(row['pos_status'])
        except:
            broken.append(row)
    return broken

In [213]:
agent_list = df.workforceid.unique().tolist()[:]

In [214]:
tdx = dx.loc[dx.workforceid == agent_list[5]]

In [215]:
not_working =[]

In [216]:
total_df = pd.DataFrame()

In [217]:
for wfid in agent_list[:]:
    if len(df.loc[df.workforceid == wfid]) < 1:
        not_working.append(wfid)
        print('woops, no df {}'.format(wfid))
        continue
    try:
        total_df = pd.concat([total_df,create_agent_timesheet(wfid)])
    except:
        not_working.append(wfid)
        print('not working')

woops, no df nan
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working
not working

In [219]:
total_df.loc[total_df.team_name == 'Kansas'].to_clipboard()

In [220]:
total_df.loc[total_df.work_verification_category == 2].sort_values(by='mins',ascending=False).to_clipboard()

In [226]:
total_df[[ 'username',
 'team_name',
 'name',
 'start_interval_log',
 'end_interval_log',
 'pos_status',
 'activitytype',
 'notes',
 'start_interval_sched',
 'end_interval_sched',
 'work_verification_category',
 'mins']].loc[(total_df.work_verification_category.isin([2])) & (total_df.mins < 300) & (total_df.mins >= 1)]

In [239]:
total_df[[ 'username',
 'team_name',
 'name',
 'start_interval_log',
 'end_interval_log',
 'pos_status',
 'activitytype',
 'notes',
 'start_interval_sched',
 'end_interval_sched',
 'work_verification_category',
 'mins']].loc[(total_df.work_verification_category.isin([2])) & (total_df.mins < 300)  & (~total_df.team_name.isin(['CSA','DGT Leads','SME','MNL Leads','COS Support','COS Agent','CST']))& (total_df.mins >= 1)].to_clipboard()

In [234]:
total_df[[ 'username',
 'team_name',
 'name',
 'start_interval_log',
 'end_interval_log',
 'pos_status',
 'activitytype',
 'notes',
 'start_interval_sched',
 'end_interval_sched',
 'work_verification_category',
 'mins']].loc[(total_df.work_verification_category.isin([2])) & (total_df.mins < 300)  & (~total_df.team_name.isin(['CSA','DGT Leads','SME','MNL Leads','COS Support','COS Agent','CST']))& (total_df.mins >= 1)][['team_name']].value_counts()

team_name     
Training          189
Texas              37
TMX Agents         21
Santa Fe           20
Cambridge          18
Little Rock        13
DR                 11
Scottsdale         10
Springfield         8
Brooklyn            7
Rochester           7
Dallas              7
El Segundo          6
Riverdale           6
Victoria            5
Alabama             5
Arizona             5
Forth Worth         4
Hollywood           4
Boston              4
Michigan            4
EL Paso             3
Nebraska            3
Alburquerque        3
Baltimore           3
TIJUANA             3
Savannah            3
Columbus            3
Reno                3
Philadelphia        3
Phoenix             3
Wisconsin           3
Hialeah             3
Connecticut         2
Indiana             2
Omaha               2
San Diego           2
Nashville           2
Delaware            2
Mesa                2
QA                  2
Florida             2
Murfreesboro        2
Knoxville           2
ST. Paul         

In [ ]:
tdf1 = total_df.groupby(['name','team_name','work_verification_category']).agg({'mins':'sum'}).reset_index()

In [ ]:
tdf1.groupby(['work_verification_category']).agg({"mins":['sum','count']}).to_clipboard()

In [ ]:
tdf1.groupby('work_verification_category').agg({'mins':['sum','count']}).to_clipboard()

In [ ]:
total_df.loc[(total_df.work_verification_category == 2) & (total_df.mins > 15) & (total_df.activitytype != 'Phone Time')]

In [ ]:
tdf1 = tdf1.pivot(index=['name','team_name'], columns='work_verification_category', values='mins').reset_index()


In [ ]:
tdf1 = tdf1.fillna(0)

In [ ]:
tdf1.to_clipboard()

In [ ]:
tdf1.to_clipboard()

In [ ]:
tdf1.loc[:, ~tdf1.columns.isin(['name', 'team_name',6,8])] = tdf1.loc[:, ~tdf1.columns.isin(['name', 'team_name',6,8])].apply(lambda row: row / row.sum(),axis=1)

In [ ]:
tdf1.to_clipboard()

In [ ]:
total_df.loc[total_df.name.str.contains("Puntero")].to_clipboard()

In [ ]:
total_df.loc[total_df.work_verification_category == "No Map"].groupby(['pos_status','activitytype'])[['mins']].sum().to_clipboard()

In [ ]:
total_df

In [ ]:
total_df.activitytype.value_counts()

In [ ]:
# Agents Scheduled for Phone Time
# Agents On Phone Time 


In [ ]:
check_time = '2024-03-08 16:45'

In [ ]:
    cri_a = total_df.start_interval_sched < check_time 
    cri_b = total_df.end_interval_sched > check_time 
    cri_status = total_df.schedule == 'Phone Time'

In [ ]:
def check_phone_time(check_time):
    cri_a = total_df.start_interval_log < check_time 
    cri_b = total_df.end_interval_log > check_time 
    cri_status = total_df.pos_status == 'Phone Time'
    phone_time = len(total_df.loc[cri_a & cri_b & cri_status])
    
    cri_a = total_df.start_interval_sched < check_time 
    cri_b = total_df.end_interval_sched > check_time 
    cri_status = total_df.schedule == 'Phone Time'
    sched_time = len(total_df.loc[cri_a & cri_b & cri_status].name.unique())
    return (phone_time, sched_time)




def check_phone_time_roster(check_time):
    cri_a = total_df.start_interval_log < check_time 
    cri_b = total_df.end_interval_log > check_time 
    
    cri_scheduled = total_df.schedule == 'Phone Time'
    cri_status = total_df.pos_status == 'Phone Time'

    return (total_df.loc[cri_a & cri_b & cri_status],total_df.loc[cri_a & cri_b & cri_scheduled])

In [ ]:
check_phone_time('2024-03-01 10:30:00')

In [ ]:
import pandas as pd

# Define start date and time
start_datetime = '2024-03-08 10:30'

# Create a DatetimeIndex starting from start_datetime with a frequency of every 5 minutes
date_series = pd.date_range(start=start_datetime, periods=5000, freq='5T')

print(date_series)


In [ ]:
dfc = pd.DataFrame(date_series)

In [ ]:
res = []

In [ ]:
for i in dfc[0].tolist():
    res.append(check_phone_time(i))

In [ ]:
dfc['on_phone'] = [x[0] for x in res]
dfc['scheduled'] = [x[1] for x in res]

In [ ]:
dfc

In [ ]:
dfc.to_clipboard()

In [ ]:
total_df.loc[total_df.name == 'Jun Reyes'].to_clipboard()

In [ ]:
tdx = df1.loc[df1.agent_names == agent]
tdx = tdx.sort_values(by='start_interval_sched').reset_index(drop=True)
tdx['last_end_interval_sched']  = tdx['end_interval_sched'].shift(1)
tdx['time_between_sched'] = [x.seconds / 60 / 60 for x in tdx.start_interval_sched - tdx['end_interval_sched'].shift(1) ]


In [ ]:
tdf = df.loc[df.name == agent_pos_key]

#### create SCHEDULED_OFF 

In [ ]:
tdx.pos_status.unique().tolist()

In [ ]:
def log_exceeds_schedule(dfdirty):
    added_rows = []
    for i,row in dfdirty.iloc[:].iterrows():
        entry1 = row.copy()
        entry1.end_interval_log = entry1.end_interval_sched
        added_rows.append(entry1)
        entry2 = row.copy()
        entry2.start_interval_log = entry2.end_interval_sched
        added_rows.append(entry2)

    df_new = pd.DataFrame(added_rows)[['name','uid','type','reason.1','start_interval_log','end_interval_log']]
    return df_new


In [ ]:
def fix_split_interval():
    tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', right_on='start_interval_sched',suffixes=('_log', '_sched'), direction='backward')
    tdf1 = tdf1[['name','uid','type','reason.1','pos_status','start_interval_log','end_interval_log','start_interval_sched','end_interval_sched']]
    tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log
    dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.pos_status))]

    dfdirty = tdf1.loc[tdf1.end_surpass]
    for i in [1,2,3]:
        df_new = log_exceeds_schedule(dfdirty)
        dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])
        dfdirty = df_new.loc[df_new.end_surpass]

    dfclean['activity_type'] = dfclean.apply(lambda row: activity_matrix.get(row.pos_status).get(row['reason.1']),axis=1)
    dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]
    dfclean[['name','reason.1','start_interval_log','end_interval_log','pos_status','start_interval_sched','end_interval_sched','mins','activity_type']].to_clipboard(index=False)

    return dfclean

#### find split Interval

In [ ]:
tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', 
    right_on='start_interval_sched',
    suffixes=('_log', '_sched'), 
    direction='backward'
)

In [ ]:
tdf1 = tdf1[['name','uid','type','reason.1','pos_status','start_interval_log','end_interval_log','start_interval_sched','end_interval_sched']]

In [ ]:
tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log

In [ ]:
dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.pos_status))]
len(dfclean)

In [ ]:
dfdirty = tdf1.loc[tdf1.end_surpass]
len(dfdirty)

In [ ]:
dfclean['activity_type'] = dfclean.apply(lambda row: activity_matrix.get(row.pos_status).get(row['reason.1']),axis=1)

In [ ]:
dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]

In [ ]:
dfclean.to_clipboard()

In [ ]:
dfclean.groupby('activity_type')['mins'].sum()

In [ ]:
dfclean[['name','reason.1','start_interval_log','end_interval_log','pos_status','start_interval_sched','end_interval_sched','mins','activity_type']].to_clipboard(index=False)

#### the problem is the end
1. make a new entry
2. clean up entry 

In [ ]:
added_rows = []
for i,row in dfdirty.iloc[:].iterrows():
    entry1 = row.copy()
    entry1.end_interval_log = entry1.end_interval_sched
    added_rows.append(entry1)
    entry2 = row.copy()
    entry2.start_interval_log = entry2.end_interval_sched
    added_rows.append(entry2)


In [ ]:
df_new = pd.DataFrame(added_rows)

In [ ]:
df_new = df_new[['name','uid','type','reason.1','start_interval_log','end_interval_log']]

In [ ]:
df_new = pd.merge_asof(df_new.sort_values(by='start_interval_log'),tdx[['pos_status','start_interval_sched','end_interval_sched']],
    left_on='start_interval_log', 
    right_on='start_interval_sched',
    suffixes=('_log', '_sched'), 
    direction='backward'
)

In [ ]:
df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log

In [ ]:
df_new.groupby('end_surpass').size()

In [ ]:
dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])

In [ ]:
dfdirty = df_new.loc[df_new.end_surpass]
len(dfdirty)

In [ ]:
def is_overlap(start1, end1, start2, end2):
    return max(start1, start2) < min(end1, end2)

In [ ]:
def get_intersection_and_non_overlap(schedule_start, schedule_end, actual_intervals):
    intersection_intervals = []
    non_overlap_intervals = []
    
    # Initial non-overlapped interval is the whole schedule
    non_overlap_intervals.append((schedule_start, schedule_end))
    
    for actual_start, actual_end in actual_intervals:
        new_non_overlap_intervals = []
        
        for non_overlap_start, non_overlap_end in non_overlap_intervals:
            # Check for overlap
            if actual_end < non_overlap_start or actual_start > non_overlap_end:
                # No overlap, keep the interval as is
                new_non_overlap_intervals.append((non_overlap_start, non_overlap_end))
            else:
                # If there's overlap, adjust non-overlapped intervals and record intersection
                if actual_start > non_overlap_start:
                    new_non_overlap_intervals.append((non_overlap_start, actual_start))
                if actual_end < non_overlap_end:
                    new_non_overlap_intervals.append((actual_end, non_overlap_end))
                
                # Add to intersection intervals
                intersection_start = max(non_overlap_start, actual_start)
                intersection_end = min(non_overlap_end, actual_end)
                intersection_intervals.append((intersection_start, intersection_end))
        
        # Update non-overlap intervals after processing this actual interval
        non_overlap_intervals = new_non_overlap_intervals

    non_compliance_seconds = 0

    for interval in non_overlap_intervals:
        non_compliance_seconds += (interval[1] - interval[0]).seconds 
    
    return intersection_intervals, non_overlap_intervals,non_compliance_seconds





In [ ]:
tdx.start_time = pd.to_datetime(tdx.start_time)
tdx.end_time = pd.to_datetime(tdx.end_time)

In [ ]:
tdx['start_interval'] = tdx.apply(startInterval,axis=1)
tdx['start_interval'] = [x.tz_localize('UTC') for x in tdx.start_interval]
tdx['end_interval'] = tdx.apply(endInterval,axis=1)
tdx['end_interval'] = [x.tz_localize('UTC') for x in tdx.end_interval]


In [ ]:
def get_non_compliance_seconds(row):
    if row.pos_status != 'Phone Time':
        return 0
    else:
        return get_intersection_and_non_overlap(row.start_interval, row.end_interval, [(row.createtime,row.endtime) for i,row in tdf.iterrows()])[2]


def get_non_compliance_seconds_stretch(row):
    if row.pos_status != 'Phone Time':
        return 0
    else:
        return get_intersection_and_non_overlap(row.start_interval, row.end_interval, [(row.createtime,row.endtime) for i,row in temp_df2.iterrows()])[2]
        

In [ ]:
tdf = tdf.loc[tdf.type == 'on']

In [ ]:
tdx['non_adherance'] = tdx.apply(lambda row: get_non_compliance_seconds(row) if row.pos_status == 'Phone Time' else 0,axis=1)

In [ ]:
tdx['non_adherance_inc'] = tdx.apply(lambda row: get_non_compliance_seconds_stretch(row) if row.pos_status == 'Phone Time' else 0,axis=1)

In [ ]:
tdx['delta'] = tdx.non_adherance - tdx.non_adherance_inc

In [ ]:
tdx.loc[tdx.pos_status == 'Phone Time'].sort_values(by='delta',ascending=False)['non_adherance'].sum() / 60 / 60

In [ ]:
tdx.loc[tdx.pos_status == 'Phone Time'].sort_values(by='delta',ascending=False)

In [ ]:
tdf